In [37]:
import os
import os.path
import sys

import pandas as pd
import numpy as np
from importlib import reload

In [38]:
LOCAL_PATH = 'data'
TRACE_NAME = 'Skype_HongKong'
RAW_TRACE = '{trace_name}.pcapng'.format(trace_name=TRACE_NAME)
TCP_TRACE_FEATURE_FILE = '{trace_name}_tcp_pkt.csv'.format(trace_name=TRACE_NAME)
UDP_TRACE_FEATURE_FILE = '{trace_name}_udp_pkt.csv'.format(trace_name=TRACE_NAME)
TCP_FLOW_FEATURE_FILE = '{trace_name}_tcp_flow.csv'.format(trace_name=TRACE_NAME)
UDP_FLOW_FEATURE_FILE = '{trace_name}_udp_flow.csv'.format(trace_name=TRACE_NAME)
# BUCKET_NAME = '' # replace with your bucket name
KEY = '' # replace with your object key

In [39]:
if not os.path.exists(os.path.join(LOCAL_PATH, RAW_TRACE)):
    if not os.path.exists(LOCAL_PATH):
        os.mkdir(LOCAL_PATH)
    
    import boto3
    import botocore

    s3 = boto3.resource('s3')

    try:
        s3.Bucket(BUCKET_NAME).download_file(KEY, os.path.join(LOCAL_PATH, RAW_TRACE))
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise

In [40]:
# convert raw trace to readable udp and tcp packet feature csv file
from python import packet_feature
reload(packet_feature)
%time packet_feature.udp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE), is_cluster=True)
%time packet_feature.tcp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE), is_cluster=True)

Conversion done
CPU times: user 3.24 s, sys: 128 ms, total: 3.37 s
Wall time: 28.8 s
Conversion done
CPU times: user 352 ms, sys: 10.1 ms, total: 362 ms
Wall time: 22.3 s


In [41]:
def to_csv(trace_df, extract_func, file, features, max_byte_per_flow):
    df = extract_func(trace_df, 1.0, upsampled=True, max_byte_per_flow=max_byte_per_flow)
    if df.shape[0] == 0:
        pd.DataFrame(columns=features).to_csv(file, index=False)
    else:
        df[features].to_csv(file, index=False)

In [42]:
from python import flow_feature
reload(flow_feature)

FEATURES = ['avg(pkt_len)', 'stddev(pkt_len)', 'fb_ratio', 'inter_arrival_time', 'pkt_count', 'duration', 'pkt_len']
MAX_BYTE_PER_FLOW = 1 * 1024 * 1024 # 1MB

tcp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))
udp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
for trace_df in [tcp_trace_df, udp_trace_df]:
    if tcp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.tcp_generate, os.path.join(LOCAL_PATH, TCP_FLOW_FEATURE_FILE), FEATURES, max_byte_per_flow=MAX_BYTE_PER_FLOW)
    elif udp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.udp_generate, os.path.join(LOCAL_PATH, UDP_FLOW_FEATURE_FILE), FEATURES, max_byte_per_flow=MAX_BYTE_PER_FLOW)
    else:
        raise

tcp flow, flow byte limit->1048576: 13027it [00:03, 3498.71it/s]          
/share/apps/python3/3.5.3/intel/lib/python3.5/site-packages/numpy-1.12.0-py3.5-linux-x86_64.egg/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/share/apps/python3/3.5.3/intel/lib/python3.5/site-packages/numpy-1.12.0-py3.5-linux-x86_64.egg/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
udp flow, flow byte limit->1048576: 365216it [02:06, 2888.60it/s]                
/home/sj2363/trace-feature-selection/python/flow_feature.py:82: RuntimeWarning: invalid value encountered in double_scalars
  flow['avg(pkt_len)'] = flow['pkt_len'] / flow['pkt_count']
/share/apps/python3/3.5.3/intel/lib/python3.5/site-packages/numpy-1.12.0-py3.5-linux-x86_64.egg/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False